In [0]:
spark.conf.set(
    
)


In [0]:
from pyspark.sql.functions import input_file_name, lit
import pyspark.sql.functions as F

# Paths
yes_path = "abfss://lakehouse@lab5mri.dfs.core.windows.net/raw/tumor_images/yes/"
no_path  = "abfss://lakehouse@lab5mri.dfs.core.windows.net/raw/tumor_images/no/"


In [0]:
yes_df = (
    spark.read.format("binaryFile")
    .load(yes_path)
    .withColumn("label", lit(1))        # tumor = yes
)


In [0]:
no_df = (
    spark.read.format("binaryFile")
    .load(no_path)
    .withColumn("label", lit(0))       # no tumor
)


In [0]:
bronze_df = yes_df.union(no_df)


In [0]:
bronze_df = bronze_df.select(
    F.col("path").alias("image_path"),
    F.col("label"),
    F.element_at(F.split(F.col("path"), "/"), -1).alias("image_id")
)

bronze_df.show(5)


+--------------------+-----+--------+
|          image_path|label|image_id|
+--------------------+-----+--------+
|abfss://lakehouse...|    1| Y24.jpg|
|abfss://lakehouse...|    1| Y41.jpg|
|abfss://lakehouse...|    1| Y91.jpg|
|abfss://lakehouse...|    1|Y103.jpg|
|abfss://lakehouse...|    1|Y108.jpg|
+--------------------+-----+--------+
only showing top 5 rows


In [0]:
bronze_df.write.mode("overwrite").parquet(
    "abfss://lakehouse@lab5mri.dfs.core.windows.net/bronze/tumor_images_raw/"
)
